# Salt Demo

In this demo notebook, we will go over how to use the salt model in Open Seismic with the F3 data block (real seismic data). Note that we will be drawing on knowledge that is introduced in the Open Seismic examples. If you have not reviewed those examples, please review them first and come back. 

## Sections
Salt.1 **Using the Default Salt Model**<br/>
Salt.2 **Using the Salt Model with Custom Model Script**<br/>
Salt.3 **Using the Salt Model with Custom Processor Scripts**

## Imports

Here are some initial imports and global variables that we need to set up before moving forward. Make sure to run `pip install -r requirements.txt` and build/pull the Docker image for Open Seismic.

In [1]:
import json
from pathlib import PurePath
import os
import numpy as np
import matplotlib.pyplot as plt

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        if 'ipynb' in root or 'ipynb' in dirs or 'ipynb' in files:
            continue
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
            
open_seismic_docker = 'open_seismic'
assets_path = PurePath(os.getcwd()).parent.joinpath('salt')
os_path = PurePath(os.getcwd()).parent.parent

models_path = os_path.joinpath('models')
#models_mount = f' -v {models_path}:/core/python/models/'
models_mount = f' -v {models_path}:/core/models/'

## Section Salt.1: Using the Default Salt Model

In this section, we will go over how to use the default salt model that exists within Open Seismic. 

### Steps
1. Construct the JSON config file for directing Open Seismic to the necessary files. For this demo, we have already constructed the JSON configuration. Run the cell below to display the JSON config file. We will be using coarse cube inference as our inference task of choice because this aligns with our intention of inference over a large piece of data.

In [2]:
json_path = assets_path.joinpath('salt_config.json')
with open(str(json_path), 'rb') as f:
    json_config = json.load(f)
print(json.dumps(json_config, indent=4))

{
    "inference_params": {
        "data": "data_mnt/Dutch_F3_data/",
        "given_model": "salt",
        "infer_type": "fine_cube_sync",
        "return_to_fullsize": "True",
        "output": "salt_demo_output",
        "benchmarking": ""
    },
    "visualize_params": {
        "input": "salt_demo_output",
        "output": "visualization",
        "model_type": "salt",
        "slice_no": "42"
    }
}


After running the cell, you might notice that we direct Open Seismic to look at the `data_mnt` directory instead of a common directory that we have recommended. The reason for this is because in this demo, we will be using the F3 data block, which a particularly large file and should not be copied multiple times. Additionally, Docker does not support symlinks on certain operating systems. Therefore, we will use one more mount command to mount the F3 data block into the right directory in Open Seismic's Docker container.

2. Construct the command to run Open Seismic.

In [3]:
local_dir = assets_path
run_dir = assets_path.joinpath('runs')
data_dir = os_path.joinpath('data', '')
os_input = '/core/mnt/salt_config.json'

infer_mount = f'-v {local_dir}:/core/mnt/'
runs_mount = f'-v {run_dir}:/core/runs/'
data_mount = f'-v {data_dir}:/core/data_mnt/'
mounts = f'{infer_mount} {data_mount} {runs_mount} {models_mount}'
os_cmd = f"docker run {mounts} {open_seismic_docker} ./run.sh -c {os_input}"
os_cmd

'docker run -v /home/sdp/ravi/open_seismic/models/salt:/core/mnt/ -v /home/sdp/ravi/open_seismic/data:/core/data_mnt/ -v /home/sdp/ravi/open_seismic/models/salt/runs:/core/runs/  -v /home/sdp/ravi/open_seismic/models:/core/models/ open_seismic ./run.sh -c /core/mnt/salt_config.json'

3. Run Open Seismic!

In [4]:
! {os_cmd}

[ARG PARSE] model= data=data_mnt/Dutch_F3_data/ output=runs/February25_07-32-00_AM_2021/salt_demo_output
[OUTPUT] Making new folder for output storage.
[INFER] Setting up inference...
[SWAP] Preprocessor not found. Defaulting to given model preprocess.
[SWAP] Postprocessor not found. Defaulting to given model postprocess.
[SWAP] Custom bin, xml, and model init not found. Using default model init with given model.
[INFO] Start benchmarking
[INFER] Benchmark output - [Step 1/11] Parsing and validating input arguments
[INFER] Benchmark output - [ WARNING ]  -nstreams default value is determined automatically for a device. Although the automatic selection usually provides a reasonable performance, but it still may be non-optimal for some cases, for more information look at README. 
[INFER] Benchmark output - [Step 2/11] Loading Inference Engine
[INFER] Benchmark output - [ INFO ] InferenceEngine:
[INFER] Benchmark output -          API version............. 2.1.2021.1.0-1237-bece22ac675-rel

## Section Salt.2: Using the Salt with Custom Model Script

In this section, we will swap out the default model script with our own custom one. You can find the model script in `demos/assets/salt_demo/only_model_script`. 

### Steps
1. Construct the JSON config file for directing Open Seismic to the necessary files. For this demo, we have already constructed the JSON configuration. Run the cell below to display the JSON config file. We will be using coarse cube inference as our inference task of choice because this aligns with our intention of inference over a large piece of data.

In [5]:
json_path = assets_path.joinpath('salt_model_config.json')
with open(str(json_path), 'rb') as f:
    json_config = json.load(f)
print(json.dumps(json_config, indent=4))

{
    "inference_params": {
        "data": "data_mnt/Dutch_F3_data/",
        "model": "models/salt/only_model_script/",
        "given_model": "salt",
        "infer_type": "fine_cube_sync",
        "output": "salt_model_demo_output"
    }
}


Notice that we have specified the model script in the `model` parameter of the JSON config file, but we have kept the `salt` option in the `given_model` parameter. In terms of priority, Open Seismic will look at the path specified in `model` first, then will look at the `given_model` files to fill in the required files.

2. Construct the command to run Open Seismic.

In [6]:
local_dir = assets_path
run_dir = assets_path.joinpath('runs')
data_dir = os_path.joinpath('data', '')
os_input = '/core/mnt/salt_model_config.json'

infer_mount = f'-v {local_dir}:/core/mnt/'
runs_mount = f'-v {run_dir}:/core/runs/'
data_mount = f'-v {data_dir}:/core/data_mnt/'
mounts = f'{infer_mount} {data_mount} {runs_mount} {models_mount}'
os_cmd = f"docker run {mounts} {open_seismic_docker} ./run.sh -c {os_input}"
os_cmd

'docker run -v /home/sdp/ravi/open_seismic/models/salt:/core/mnt/ -v /home/sdp/ravi/open_seismic/data:/core/data_mnt/ -v /home/sdp/ravi/open_seismic/models/salt/runs:/core/runs/  -v /home/sdp/ravi/open_seismic/models:/core/models/ open_seismic ./run.sh -c /core/mnt/salt_model_config.json'

3. Run Open Seismic!

In [7]:
! {os_cmd}

[ARG PARSE] model=models/salt/only_model_script/ data=data_mnt/Dutch_F3_data/ output=runs/February25_07-40-06_AM_2021/salt_model_demo_output
[OUTPUT] Making new folder for output storage.
[INFER] Setting up inference...
[SWAP] Preprocessor not found. Defaulting to given model preprocess.
[SWAP] Postprocessor not found. Defaulting to given model postprocess.
[SWAP] Found custom model script.
[SWAP] Custom bin or xml not found. Using given model xml and bin with custom model init.
[INFER] Using model: salt_model
[LOADER] Loading file: data_mnt/Dutch_F3_data//f3_8bit.segy
[INFER] Conducting inference...
[INFER] Conducting inference on input: f3_8bit.segy...
100%|##########| 71340/71340 [02:06<00:00, 562.96it/s] [INFER] Saving output to output path: runs/February25_07-40-06_AM_2021/salt_model_demo_output/f3_8bit.segy-input/out.npy
[INFER] Complete!



## Section Salt.3: Using the Salt with Custom Processor Scripts

In this section, we will swap out the default processor scripts with our own custom ones. You can find the model script in `demos/assets/salt_demo/only_processor_scripts`. 

### Steps
1. Construct the JSON config file for directing Open Seismic to the necessary files. For this demo, we have already constructed the JSON configuration. Run the cell below to display the JSON config file. We will be using coarse cube inference as our inference task of choice because this aligns with our intention of inference over a large piece of data.

In [8]:
json_path = assets_path.joinpath('salt_proc_config.json')
with open(str(json_path), 'rb') as f:
    json_config = json.load(f)
print(json.dumps(json_config, indent=4))

{
    "inference_params": {
        "data": "data_mnt/Dutch_F3_data/",
        "model": "mnt/only_processor_scripts/",
        "given_model": "salt",
        "infer_type": "fine_cube_sync",
        "output": "salt_proc_demo_output"
    }
}


Notice that we have specified the model script in the `model` parameter of the JSON config file, but we have kept the `salt` option in the `given_model` parameter. In terms of priority, Open Seismic will look at the path specified in `model` first, then will look at the `given_model` files to fill in the required files.

2. Construct the command to run Open Seismic.

In [9]:
local_dir = assets_path
run_dir = assets_path.joinpath('runs')
data_dir = os_path.joinpath('data', '')
os_input = '/core/mnt/salt_proc_config.json'

infer_mount = f'-v {local_dir}:/core/mnt/'
runs_mount = f'-v {run_dir}:/core/runs/'
data_mount = f'-v {data_dir}:/core/data_mnt/'
mounts = f'{infer_mount} {data_mount} {runs_mount} {models_mount}'
os_cmd = f"docker run {mounts} {open_seismic_docker} ./run.sh -c {os_input}"
os_cmd

'docker run -v /home/sdp/ravi/open_seismic/models/salt:/core/mnt/ -v /home/sdp/ravi/open_seismic/data:/core/data_mnt/ -v /home/sdp/ravi/open_seismic/models/salt/runs:/core/runs/  -v /home/sdp/ravi/open_seismic/models:/core/models/ open_seismic ./run.sh -c /core/mnt/salt_proc_config.json'

3. Run Open Seismic!

In [10]:
! {os_cmd}

[ARG PARSE] model=mnt/only_processor_scripts/ data=data_mnt/Dutch_F3_data/ output=runs/February25_07-42-21_AM_2021/salt_proc_demo_output
[OUTPUT] Making new folder for output storage.
[INFER] Setting up inference...
[SWAP] Found custom preporcessing script.
[SWAP] Found custom postporcessing script.
[SWAP] Custom bin, xml, and model init not found. Using default model init with given model.
[INFER] Using model: salt_model
[LOADER] Loading file: data_mnt/Dutch_F3_data//f3_8bit.segy
[INFER] Conducting inference...
[INFER] Conducting inference on input: f3_8bit.segy...
100%|##########| 71340/71340 [02:04<00:00, 572.25it/s] [INFER] Saving output to output path: runs/February25_07-42-21_AM_2021/salt_proc_demo_output/f3_8bit.segy-input/out.npy
[INFER] Complete!



Congratulations! You now know how to use the salt model in Open Seismic.

## Summary
In this demo, you learned about:
1. Using the default salt model with your data
2. Using the salt model with your data and model script
3. Using the salt model with your data and processor scripts

We haven't talked about combining custom model and processor scripts together, but this can be done as well by storing the model and processor scripts in one mount directory. If you would like to learn more about the other models, please go to their respective folders and run their demos.